In [1]:
# Run below commands in google colab
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
# download spark3.0.0
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz   
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [3]:
# unzip it
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# install findspark 
!pip install -q findspark

In [4]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [5]:
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar

--2020-12-21 16:34:37--  https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231556205 (221M) [application/java-archive]
Saving to: ‘xgboost4j_3.0-1.0.0-0.1.0.jar’

xgboost4j_3.0-1.0.0 100%[===================>] 220.83M   146MB/s    in 1.5s    

2020-12-21 16:34:39 (146 MB/s) - ‘xgboost4j_3.0-1.0.0-0.1.0.jar’ saved [231556205/231556205]

--2020-12-21 16:34:39--  https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2040779 (1.9M) [application/java-archive]
S

In [6]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/xgboost4j_3.0-1.0.0-0.1.0.jar,/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar pyspark-shell'

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.addPyFile('/content/xgboost4j_3.0-1.0.0-0.1.0.jar')
spark.sparkContext.addPyFile('/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar')

In [8]:
reader = spark.read

In [9]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import CrossValidator

import numpy as np
import pandas as pd

In [10]:
from pyspark.sql.types import FloatType, IntegerType, StructField, StructType

In [11]:
!wget https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_train.parquet
!wget https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_test.parquet

import pyarrow.parquet as pq
train_data = reader.format('parquet').load('covtype_train.parquet')
test_data = reader.format('parquet').load('covtype_test.parquet')
pq_file=pq.read_table('covtype_train.parquet')
label="target"
features = [ x for x in pq_file.column_names if x != label ]

--2020-12-21 16:34:47--  https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_train.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6827427 (6.5M) [application/octet-stream]
Saving to: ‘covtype_train.parquet’

covtype_train.parqu 100%[===================>]   6.51M  25.1MB/s    in 0.3s    

2020-12-21 16:34:47 (25.1 MB/s) - ‘covtype_train.parquet’ saved [6827427/6827427]

--2020-12-21 16:34:47--  https://raw.githubusercontent.com/sauravdev/Spark_3_GPU_CPU/master/covtype_test.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response

In [12]:
train_data.show()
train_data.count()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|Wilder

464810

In [13]:


from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

def vectorize(data_frame):
    to_floats = [ col(x.name).cast(FloatType()) for x in data_frame.schema ]
    return (VectorAssembler()
        .setInputCols(features)
        .setOutputCol('features')
        .transform(data_frame.select(to_floats))
        .select(col('features'), col(label)))

train_data = vectorize(train_data)
test_data = vectorize(test_data)

In [14]:
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'hist',
    'maxDepth': 6, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'verbosity': 2
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCol('features')

In [15]:
%%time
model_cpu_spark=classifier.fit(train_data)

CPU times: user 75.6 ms, sys: 22.5 ms, total: 98.1 ms
Wall time: 2min 26s


In [16]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# Make predictions.
predictions = model_cpu_spark.transform(test_data)
predictions.show()
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy is %g" % accuracy)
print("Test Error = %g" % (1.0 - accuracy))

predictions.select('prediction').distinct().collect()

+--------------------+------+--------------------+--------------------+----------+
|            features|target|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+----------+
|(54,[0,1,2,3,4,5,...|   1.0|[0.7180565595626831]|[0.7180565595626831]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0| [3.123528242111206]| [3.123528242111206]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0| [2.635711669921875]| [2.635711669921875]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0|[3.2950150966644287]|[3.2950150966644287]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0|  [3.11338210105896]|  [3.11338210105896]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0|[1.9348220825195312]|[1.9348220825195312]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0|[1.3748362064361572]|[1.3748362064361572]|       0.0|
|(54,[0,1,2,3,4,5,...|   4.0|[2.5657737255096436]|[2.5657737255096436]|       0.0|
|(54,[0,1,2,3,4,5,...|   1.0|[0.9863039255142212]|[0.9863039255142212]|       0.0|
|(54

[Row(prediction=0.0)]

In [17]:
spark.stop()

In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [19]:
x_df_train = pd.read_parquet('covtype_train.parquet', engine='pyarrow')
X_train = x_df_train.loc[:, x_df_train.columns != 'target']
y_train = x_df_train["target"]
x_df_test = pd.read_parquet('covtype_test.parquet', engine='pyarrow')
X_test = x_df_test.loc[:, x_df_test.columns != 'target']
y_test = x_df_test["target"]


In [20]:
%%time

model = XGBClassifier( eta= 0.1,
    gamma= 0.1,
    missing= 0.0,
    treeMethod= 'hist',
    maxDepth= 6, 
    growPolicy= 'depthwise',
    lambda_= 1.0,
    subsample= 1.0,
    numRound= 1000,
    verbosity= 2)
model.fit(X_train,y_train)

[16:38:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:38:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:38:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[16:38:56] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:38:57] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:38:57] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:38:58] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[16:38:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nod

In [21]:
sk_pred = model.predict(X_test)
sk_pred = np.round(sk_pred)
sk_acc = round(accuracy_score(y_test, sk_pred), 2)
print("XGB accuracy using Sklearn:", sk_acc*100, '%')

XGB accuracy using Sklearn: 74.0 %
